In [2]:
!pip3 install datasets
!pip3 install transformers
!pip3 install evaluate
!pip3 install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 65.4 MB/s eta 0:00

In [3]:
from transformers import TrainingArguments, Trainer
from transformers import pipeline
from tqdm import tqdm
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import evaluate
import pandas as pd
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset

In [4]:
dataset = load_dataset("xsum")
splitLengths = [len(dataset[split]) for split in dataset]
print(f"Split lengths: {splitLengths}")
print(f"Columns: {dataset['train'].column_names}")
print(f"\nDocument:")
print(dataset["test"][0]["document"])
print("\nSummary")
print(dataset["test"][0]["summary"])

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Split lengths: [204045, 11332, 11334]
Columns: ['document', 'summary', 'id']

Document:
Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.
Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.
The Welsh Government said more people than ever were getting help to address housing problems.
Changes to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.
Prison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.
However, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.
Andrew Stevens, who works in

In [5]:
summerizationPipeLine = pipeline("summarization", model="google/pegasus-xsum", framework='pt')
maxLength = 512
inputText = dataset["test"][0]["document"][:maxLength]
print(inputText)
summerizationPipeLineOutput = summerizationPipeLine(inputText)
print("Summary:")
print(summerizationPipeLineOutput[0]["summary_text"].replace(" .<n>", ".\n"))

Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.
Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.
The Welsh Government said more people than ever were getting help to address housing problems.
Changes to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.
Prison Link Cymru, which 
Summary:
More than 1,000 homeless people were referred to a charity in Wales last year.


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

def chunks(list_of_elements, batch_size):
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def summarize_and_evaluate(dataset, metric, model, tokenizer,
                       batch_size=16, device=device,
                       column_text="article", column_summary="highlights"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True,padding="max_length", return_tensors="pt")
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),attention_mask=inputs["attention_mask"].to(device),length_penalty=0.8, num_beams=8, max_length=128)
        summaryArray = [tokenizer.decode(s, skip_special_tokens=True,clean_up_tokenization_spaces=True)for s in summaries]
        summaryArray = [d.replace("<n>", " ") for d in summaryArray]
    return metric.compute(predictions=summaryArray, references=target_batch)

In [8]:
modelForFineTuning = "ainize/bart-base-cnn"
tokenizer = AutoTokenizer.from_pretrained(modelForFineTuning)
model = AutoModelForSeq2SeqLM.from_pretrained(modelForFineTuning).to(device)

In [ ]:
rougeMetric = evaluate.load("rouge")
score = summarize_and_evaluate(dataset["test"], rougeMetric, model,tokenizer, column_text="document",column_summary="summary", batch_size=8)

100%|██████████| 1417/1417 [56:36<00:00,  2.40s/it]


In [ ]:
pd.DataFrame(score, index=["bart"])

,rouge1,rouge2,rougeL,rougeLsum
bart,0.190064,0.021406,0.115531,0.145851


In [9]:
def preprocess_batch(example_batch, tokenizer):
    inputs = tokenizer(example_batch["document"], truncation=True, max_length=1024)
    targets = tokenizer(example_batch["summary"], truncation=True, max_length=128)
    return {"input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "labels": targets["input_ids"]}

tokenized_dataset = dataset.map(lambda examples: preprocess_batch(examples, tokenizer), batched=True)
torch_dataset = tokenized_dataset.remove_columns(["document", "summary"]).with_format("torch")

Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
seq2seq_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
trainingArgs = TrainingArguments(
    output_dir="bart-samsum", num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10, push_to_hub=False,
    evaluation_strategy="steps", eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16)

In [ ]:
trainer = Trainer(model=model, args=trainingArgs,tokenizer=tokenizer, data_collator=seq2seq_collator,
                  train_dataset=torch_dataset["train"],eval_dataset=torch_dataset["validation"])
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,2.305900,2.070141
1000,2.144800,2.018808
1500,2.288700,1.981193
2000,2.107000,1.955153
2500,2.180000,1.937821
3000,2.042700,1.924983
3500,2.190000,1.911012
4000,2.074400,1.903477
4500,2.076200,1.886719
5000,2.154500,1.872762


TrainOutput(global_step=12752, training_loss=2.0462032857174557, metrics={'train_runtime': 16225.5345, 'train_samples_per_second': 12.576, 'train_steps_per_second': 0.786, 'total_flos': 5.54832220819968e+16, 'train_loss': 2.0462032857174557, 'epoch': 1.0})

In [ ]:
score = summarize_and_evaluate(dataset["test"], rougeMetric, trainer.model, tokenizer,batch_size=2, column_text="document", column_summary="summary")
pd.DataFrame(score, index=[f"bart_finetuned"])

100%|██████████| 5667/5667 [40:11<00:00,  2.35it/s]


,rouge1,rouge2,rougeL,rougeLsum
bart_finetuned,0.299471,0.137529,0.178307,0.178307


In [ ]:
testText = dataset["test"][0]["document"]
inputs = tokenizer(testText, max_length=1024, truncation=True,padding="max_length", return_tensors="pt")
summaries = model.generate(input_ids=inputs["input_ids"].to(device),attention_mask=inputs["attention_mask"].to(device),length_penalty=0.8, num_beams=8, max_length=128)
summaryArray = [tokenizer.decode(s, skip_special_tokens=True,clean_up_tokenization_spaces=True)for s in summaries]
summaryArray = [d.replace("<n>", " ") for d in summaryArray]

In [ ]:
print(summaryArray)

['There is a "desperate need" for one-bedroom flats for homeless people, a charity has said.']
